# 튜토리얼 5 - 주식 매매 백테스트

과거의 주가정보를 이용하여 특정한 주문전략을 수행하였을 때의 성과를 계산하는 것을 백테스트(backtest)라고 합니다.

kquant 패키지는 개별 주식 및 주식 포트폴리오에 대한 백테스트(backtest) 기능을 제공합니다.

본 튜토리얼에서는 개별 주식에 대해 매매 백테스트하는 법을 설명합니다.

In [1]:
import kquant as kq

## 5.1 백테스트용 입력정보 준비

주식매매 백테스트를 하기 위해서는 날짜별 주식 주문 정보가 담긴 pandas 데이터프레임 `df_order`을 준비해야 합니다. 
`df_order`은 다음과 같은 2개의 열을 가져야 합니다.

- `DATE`: 주문하는 주식의 날짜 정보 리스트. 중복된 날짜가 있으면 안되고 pandas `to_datetime` 함수로 변환가능한 `"2023-01-01` 형식 등의 문자열 
- `ORDER`: 주문하는 주식의 수량 리스트. 정수만 가능하며 양수인 경우 매수, 음수인 경우 매도로 처리함


예를 들어 다음과 같이 2023년 1월 2일에 삼성전자 주식을 10주 매수하고 11일에 10주 매도하는 주문정보를 만들 수 있습니다.

In [2]:
import pandas as pd

df_order = pd.DataFrame({
    "DATE": ["2023-01-02", "2023-01-11"],
    "ORDER": [10, -10]
})

df_order


,DATE,ORDER
0,2023-01-02,10
1,2023-01-11,-10


## 5.2 백테스트 실시

주문 정보 데이터프레임이 준비되면 `backtest_stock_daily` 함수를 사용하여 주식 매매 백테스트를 할 수 있습니다. 

`backtest_stock_daily` 함수는 기본적으로 다음과 같은 입력 인수를 받습니다.

- `symbol`: 종목 단축코드 문자열
- `order`: 날짜별 주식 주문 정보가 담긴 pandas 데이터프레임.
- `start_date` (옵션): 백테스트 시작 날짜 정보. 입력하지 않으면 `df_order` 데이터프레임의 첫 날짜를 사용.
- `end_date` (옵션): 백테스트 시작 날짜 정보. 입력하지 않으면 `df_order` 데이터프레임의 마지막 날짜를 사용.
- `init_cash` (옵션): 초기보유 현금. 반드시 이름있는 인수(named parameter) 형태로 입력해야 함.

`backtest_stock_daily` 함수에 `df_order` 데이터프레임을 입력하여 실행하면 백테스트를 수행하고 결과를 담은 `df_result` 데이터프레임을 출력합니다.

In [3]:
df_result = kq.backtest_stock_daily("005930", df_order, init_cash=1_000_000)

[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-11] 종목: 005930, 주문전 보유수량:     10 주문수량:    -10, 매매수량:    -10, 주문후 보유수량:      0


## 5.3 백테스트 결과 정보

백테스트 결과를 담은 데이터프레임 `df_result`에는 백테스트 기간동안의 다음 정보를 담고 있습니다.

- DATE: 날짜
- SYMBOL: 종목단축코드
- PRICE: 주식 평가를 위한 당일 종가
- ORDER: 주문수량, 양수이면 매수, 음수이면 매도
- QTY: 실제 매매수량, 양수이면 매수, 음수이면 매도
- TRADE_PRICE: 체결 가격, 매매일 종가에서 슬리피지(slippage) 비율만큼 손실을 보면서 체결
- FEE: 증권사 및 유관기관 수수료 금액
- TRADE_TAX: 매도시 발생하는 증권거래세 금액
- SLIPPAGE: 주식의 현재 가격과 실제 매매 가격의 차이에 의해 발생하는 슬리피지(slippage)
- CASHFLOW: 현금흐름, 양수이면 매도시 발생하는 현금유입, 음수이면 매수시 발생하는 현금유출
- CASH: 당일의 보유 현금 금액
- POSITION: 당일의 보유 주식 수량
- AVG_PRICE: 당일기준 보유 주식의 역사적 평균가격, 선입선출 방식으로 계산
- HIST_VALUE: 보유 주식의 매수 금액, 선입선출 방식으로 계산
- STOCK_VALUE: 당일의 주식 평가액
- TOTAL_VALUE: 당일의 주식 평가액과 현금 보유액의 합계
- REAL_PROFIT: 주식 매도시 발생하는 실현손익, 음수이면 손실
- UNREAL_PROFIT: 보유 주식에 대한 평가손익, 음수이면 손실
- PROFIT: 총손익, 누적 실현손익과 최종 평가손익의 합, 음수이면 손실

예를 들어 실제 매매수량 `QTY`는 이후에 설명할 여러가지 주문 오류로 인해 실제 주문수량 `ORDER`와 다를 수 있습니다.

In [4]:
df_result

,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,005930,"55,500",10,10,"55,500",10,"55,500.0000",0,0,0,"-555,000","445,000","555,000","555,000","1,000,000",0,0,0,"1,000,000",0
1,2023-01-03,005930,"55,400",0,0,0,10,"55,500.0000",0,0,0,0,"445,000","555,000","554,000","999,000",0,"-1,000","-1,000","1,000,000","1,000"
2,2023-01-04,005930,"57,800",0,0,0,10,"55,500.0000",0,0,0,0,"445,000","555,000","578,000","1,023,000",0,"23,000","23,000","1,023,000",0
3,2023-01-05,005930,"58,200",0,0,0,10,"55,500.0000",0,0,0,0,"445,000","555,000","582,000","1,027,000",0,"27,000","27,000","1,027,000",0
4,2023-01-06,005930,"59,000",0,0,0,10,"55,500.0000",0,0,0,0,"445,000","555,000","590,000","1,035,000",0,"35,000","35,000","1,035,000",0
5,2023-01-09,005930,"60,700",0,0,0,10,"55,500.0000",0,0,0,0,"445,000","555,000","607,000","1,052,000",0,"52,000","52,000","1,052,000",0
6,2023-01-10,005930,"60,400",0,0,0,10,"55,500.0000",0,0,0,0,"445,000","555,000","604,000","1,049,000",0,"49,000","49,000","1,052,000","3,000"
7,2023-01-11,005930,"60,500",-10,-10,"60,500",0,0.0000,0,0,0,"605,000","1,050,000",0,0,"1,050,000","50,000",0,"50,000","1,052,000","2,000"


위 백테스트 결과로부터 1월 2일 55,500원에 10주 매수하고 11일 60,500원에 전량 매도하여 50,000원의 수익을 얻었음을 알 수 있습니다.

In [5]:
print(
    f"매수가격: {df_result.TRADE_PRICE.iloc[0]:,}원, "
    f"매도가격: {df_result.TRADE_PRICE.iloc[-1]:,}원, " 
    f"수익: {df_result.PROFIT.iloc[-1]:,}원"
)

매수가격: 55,500원, 매도가격: 60,500원, 수익: 50,000원


## 5.4 백테스트 결과 시각화

kquant 패키지는 백테스트 결과 시각화를 위한 `backtest_plot_stock_daily` 함수를 제공합니다. 인수로는 백테스트 결과 데이터프레임을 받습니다. 

In [6]:
kq.backtest_plot_stock_daily(df_result)

## 5.5 백테스트 성능 평가

`backtest_stats_stock_daily` 함수를 사용하면 백테스트의 성능을 다양한 통계수치로 볼 수 있습니다.
성능평가 항목은 다음과 같습니다.

- START_DATE : 백테스트 시작일
- END_DATE : 백테스트 종료일
- DAYS : 백테스트 기간
- START_PRICE : 백테스트 시작일 가격
- END_PRICE : 백테스트 종료일 가격
- BENCHMARK_RETURN : 백테스트 기간동안의 단순 주식 수익률
- INIT_CASH : 초기보유 현금
- START_VALUE : 백테스트 시작일 총자산 가치
- END_VALUE : 백테스트 종료일 총자산 가치
- MAX_VALUE : 백테스트 기간중 총자산 가치의 최고치
- MAX_VALUE_DATE : 백테스트 기간중 총자산 가치의 최고치 기록일
- MIN_VALUE : 백테스트 기간중 총자산 가치의 최저치
- MIN_VALUE_DATE:  : 백테스트 기간중 총자산 가치의 최저치 기록일
- PROFIT : 수익
- TOTAL_RETURN : 초기보유 현금에 대한 수익률
- ANNUALIZED_RETURN : 연율화한 수익률
- VOLATILITY : 변동성
- SHARPE_RATIO : 샤프지수
- TOTAL_FEE : 수수료 비용 합계
- TOTAL_TRADE_TAX : 매도세 비용 합계
- TOTAL_SLIPPAGE : 슬리피지 비용 합계
- TOTAL_COST : 전체 비용 합계
- WINNING_TRADE_COUNT : 실현수익이 양수인 횟수
- LOSING_TRADE_COUNT : 실현수익이 음수인 횟수
- WIN_RATE : 전체 매도 횟수 중 실현수익이 양수인 횟수의 비율
- WINNING_PL_SUM : 실현수익이 양수인 경우의 수익 합계
- LOSING_PL_SUM : 실현수익이 음수인 경우의 수익 합계
- WINNING_PL_AVG : 실현수익이 양수인 경우의 수익 평균
- LOSING_PL_AVG : 실현수익이 양수인 경우의 수익 평균
- MAXDRAWDOWN : 맥시멈 드로운다운(Maximum Draw-down: 최고 자산가치 대비 하락)
- MAXDRAWDOWN_DATE : 맥시멈 드로운다운 날짜

In [7]:
kq.backtest_stats_stock_daily(df_result)

START_DATE             2023-01-02 00:00:00
END_DATE               2023-01-11 00:00:00
DAYS                                     8
START_PRICE                          55500
END_PRICE                            60500
BENCHMARK_RETURN                    0.0901
INIT_CASH                          1000000
START_VALUE                        1000000
END_VALUE                          1050000
MAX_VALUE                          1052000
MAX_VALUE_DATE         2023-01-09 00:00:00
MIN_VALUE                           999000
MIN_VALUE_DATE         2023-01-03 00:00:00
PROFIT                               50000
TOTAL_RETURN                        0.0500
ANNUALIZED_RETURN                   1.5750
VOLATILITY                          0.1569
SHARPE_RATIO                       10.0357
TOTAL_FEE                                0
TOTAL_TRADE_TAX                          0
TOTAL_SLIPPAGE                           0
TOTAL_COST                               0
WINNING_TRADE_COUNT                      1
LOSING_TRAD

## 5.6 백테스트 날짜 설정

주문정보 데이터프레임의 날짜와 상관없이 백테스트 기간을 설정하고 싶은 경우에는 `start_date` 및 `start_date` 인수를 주면 됩니다.

In [8]:
df_result2 = kq.backtest_stock_daily(
    "005930", 
    df_order, 
    "2022-12-21", 
    "2023-01-21", 
    init_cash=1_000_000)
df_result2

[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-11] 종목: 005930, 주문전 보유수량:     10 주문수량:    -10, 매매수량:    -10, 주문후 보유수량:      0


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2022-12-21,005930,"58,000",0,0,0,0,0.0000,0,0,0,0,"1,000,000",0,0,"1,000,000",0,0,0,"1,000,000",0
1,2022-12-22,005930,"59,100",0,0,0,0,0.0000,0,0,0,0,"1,000,000",0,0,"1,000,000",0,0,0,"1,000,000",0
2,2022-12-23,005930,"58,100",0,0,0,0,0.0000,0,0,0,0,"1,000,000",0,0,"1,000,000",0,0,0,"1,000,000",0
3,2022-12-26,005930,"57,900",0,0,0,0,0.0000,0,0,0,0,"1,000,000",0,0,"1,000,000",0,0,0,"1,000,000",0
4,2022-12-27,005930,"58,100",0,0,0,0,0.0000,0,0,0,0,"1,000,000",0,0,"1,000,000",0,0,0,"1,000,000",0
5,2022-12-28,005930,"56,600",0,0,0,0,0.0000,0,0,0,0,"1,000,000",0,0,"1,000,000",0,0,0,"1,000,000",0
6,2022-12-29,005930,"55,300",0,0,0,0,0.0000,0,0,0,0,"1,000,000",0,0,"1,000,000",0,0,0,"1,000,000",0
7,2023-01-02,005930,"55,500",10,10,"55,500",10,"55,500.0000",0,0,0,"-555,000","445,000","555,000","555,000","1,000,000",0,0,0,"1,000,000",0
8,2023-01-03,005930,"55,400",0,0,0,10,"55,500.0000",0,0,0,0,"445,000","555,000","554,000","999,000",0,"-1,000","-1,000","1,000,000","1,000"
9,2023-01-04,005930,"57,800",0,0,0,10,"55,500.0000",0,0,0,0,"445,000","555,000","578,000","1,023,000",0,"23,000","23,000","1,023,000",0


In [9]:
kq.backtest_plot_stock_daily(df_result2)

## 5.7 수수료 및 슬리피지

보다 현실적인 백테스트를 위해 `backtest_stock_daily` 함수 사용시 
다음 인수로 수수료 및 슬리피지(slippage)를 설정할 수 있습니다.
모든 인수는 이름있는 인수(named parameter) 형식으로 주어야 합니다.

- `broker_fee_percent`: 증권사 수수료(%)
- `exchange_fee_percent`: 유관기관 수수료(%)
- `trade_tax_percent`: 매도시 주식양도세(%)
- `slippage_tick`: 거래시 발생하는 슬리피지(slippage)틱(호가가격단위)

위의 모든 인수는 %(percent) 단위이며 소수점 이하 자리수가 발생하는 경우 올림하여 처리합니다. 

예를 들어 증권사 수수료가 0.015%, 유관기관 수수료가 0.0036396%, 매도시 주식양도세가 0.20%이고
슬리피지가 1틱인 경우에는 다음과 같습니다.

In [10]:
df_result3 = kq.backtest_stock_daily(
    "005930",
    df_order,
    init_cash=1_000_000,
    broker_fee_percent=0.015,
    exchange_fee_percent=0.0036396,
    trade_tax_percent=0.20,
    slippage_tick=1,
)

df_result3


[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-11] 종목: 005930, 주문전 보유수량:     10 주문수량:    -10, 매매수량:    -10, 주문후 보유수량:      0


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,005930,"55,500",10,10,"55,600",10,"55,600.0000",105,0,"1,000","-556,105","443,895","556,000","555,000","998,895",-105,"-1,000","-1,105","1,000,000","1,105"
1,2023-01-03,005930,"55,400",0,0,0,10,"55,600.0000",0,0,0,0,"443,895","556,000","554,000","997,895",0,"-2,000","-2,105","1,000,000","2,105"
2,2023-01-04,005930,"57,800",0,0,0,10,"55,600.0000",0,0,0,0,"443,895","556,000","578,000","1,021,895",0,"22,000","21,895","1,021,895",0
3,2023-01-05,005930,"58,200",0,0,0,10,"55,600.0000",0,0,0,0,"443,895","556,000","582,000","1,025,895",0,"26,000","25,895","1,025,895",0
4,2023-01-06,005930,"59,000",0,0,0,10,"55,600.0000",0,0,0,0,"443,895","556,000","590,000","1,033,895",0,"34,000","33,895","1,033,895",0
5,2023-01-09,005930,"60,700",0,0,0,10,"55,600.0000",0,0,0,0,"443,895","556,000","607,000","1,050,895",0,"51,000","50,895","1,050,895",0
6,2023-01-10,005930,"60,400",0,0,0,10,"55,600.0000",0,0,0,0,"443,895","556,000","604,000","1,047,895",0,"48,000","47,895","1,050,895","3,000"
7,2023-01-11,005930,"60,500",-10,-10,"60,400",0,0.0000,113,"1,208","1,000","602,679","1,046,574",0,0,"1,046,574","46,679",0,"46,574","1,050,895","4,321"


백테스트 결과에서 총 3,426원의 비용이 발생하여 수익이 50,000원이 아닌 46,574원이 되었음을 알 수 있습니다.

In [11]:
fee_buy = df_result3.FEE.iloc[0]
fee_sell = df_result3.FEE.iloc[-1]
fee = fee_buy + fee_sell
tax = df_result3.TRADE_TAX.iloc[-1]
slippage_buy = df_result3.SLIPPAGE.iloc[0]
slippage_sell = df_result3.SLIPPAGE.iloc[-1]
slippage = slippage_buy + slippage_buy
price_buy = df_result3.TRADE_PRICE.iloc[0]
price_sell = df_result3.TRADE_PRICE.iloc[-1]
simple_profit = (price_sell - price_buy) * 10
profit = df_result3.PROFIT.iloc[-1]

print(f"""
- 증권사 및 유관기관 수수료 {fee}원(매수시 {fee_buy}원, 매도시 {fee_sell}원)
- 매도세 {tax:,}원
- 슬리피지 {slippage:,}원 (매수시 {slippage_buy:,}원 + 매도시 {slippage_sell:,}원)
- 총비용 {fee + tax + slippage:,}원
- 비용제외수익 {simple_profit:,}원 = ({price_sell:,} - {price_buy:,}) x 10 = {price_sell - price_buy:,} x 10
- 총수익 {profit:,}원 = 비용제외수익 {simple_profit:,}원 - 총비용 {fee + tax + slippage:,}원
""")



- 증권사 및 유관기관 수수료 218원(매수시 105원, 매도시 113원)
- 매도세 1,208원
- 슬리피지 2,000원 (매수시 1,000원 + 매도시 1,000원)
- 총비용 3,426원
- 비용제외수익 48,000원 = (60,400 - 55,600) x 10 = 4,800 x 10
- 총수익 46,574원 = 비용제외수익 48,000원 - 총비용 3,426원



## 5.8 실현손익, 평가손익, 총손익

백테스트 결과에서 실현손익 `REAL_PROFIT`은 다음과 같이 계산합니다.

- 주식매수시 매수수수료합계(음수)
- 주식매도시 선입선출법(FIFO: First-In First-Out)으로 계산한 수익에서 매도세와 매도수수료를 차감한 금액

즉, 주식매도시에는 먼저 매수한 주식부터 순차적으로 매도하는 방식으로 매수가를 계산하여 매도가로부터 차감하여 수익을 계산합니다.

In [12]:
df_order4 = pd.DataFrame({
    "DATE": ["2023-01-02", "2023-01-03", "2023-01-04"],
    "ORDER": [10, 20, -15]
})

df_order4


,DATE,ORDER
0,2023-01-02,10
1,2023-01-03,20
2,2023-01-04,-15


In [13]:
df_result4 = kq.backtest_stock_daily(
    "005930",
    df_order4, 
    init_cash=10_000_000,
    broker_fee_percent=0.015, 
    exchange_fee_percent=0.0036396,
    trade_tax_percent=0.23,
)

df_result4


[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-03] 종목: 005930, 주문전 보유수량:     10 주문수량:     20, 매매수량:     20, 주문후 보유수량:     30
[2023-01-04] 종목: 005930, 주문전 보유수량:     30 주문수량:    -15, 매매수량:    -15, 주문후 보유수량:     15


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,005930,"55,500",10,10,"55,500",10,"55,500.0000",105,0,0,"-555,105","9,444,895","555,000","555,000","9,999,895",-105,0,-105,"10,000,000",105
1,2023-01-03,005930,"55,400",20,20,"55,400",30,"55,433.3333",208,0,0,"-1,108,208","8,336,687","1,663,000","1,662,000","9,998,687",-208,"-1,000","-1,313","10,000,000","1,313"
2,2023-01-04,005930,"57,800",-15,-15,"57,800",15,"55,400.0000",163,"1,995",0,"864,842","9,201,529","831,000","867,000","10,068,529","32,842","36,000","68,529","10,068,529",0


In [14]:
r0 = df_result4.iloc[0]
r1 = df_result4.iloc[1]
r2 = df_result4.iloc[2]
f0 = r0.FEE
f1 = r1.FEE
f2 = r2.FEE
tax = r2.TRADE_TAX
cf_sell = r2.TRADE_PRICE * 15
cf_buy1 = r0.TRADE_PRICE * 10
cf_buy2 = r1.TRADE_PRICE * 5
cf_buy = cf_buy1 + cf_buy2
r_profit = r2.REAL_PROFIT
print(f"""
- 1차 매수금액 {cf_buy1:,}원 = 매수주가 {r0.TRADE_PRICE:,}원 x 매도수량 {abs(r0.QTY):,}주
- 2차 매수금액 {cf_buy2:,}원 = 매수주가 {r1.TRADE_PRICE:,}원 x 매도수량 5주
- 총매수금액 {cf_buy:,}원 = {cf_buy1:,}원 + {cf_buy2:,}원
- 총매도금액 {cf_sell:,}원 = 매도주가 {r2.TRADE_PRICE:,}원 x 매도수량 {abs(r2.QTY):,}주 
- 매도세 {tax:,}원 = 총매도금액 {cf_sell:,}원 x 0.23% 
- 매도수수료 {f2:,}원 = 총매도금액 {cf_sell:,}원 x 0.015% + 총매도금액 {cf_sell:,}원 x 0.0036396% 
- 총실현수익 {r_profit:,}원 = 총매도금액 {cf_sell:,}원 - 총매수금액 {cf_buy:,}원 - 매도세 {tax:,}원 - 매도수수료 {f2:,}원
""")



- 1차 매수금액 555,000원 = 매수주가 55,500원 x 매도수량 10주
- 2차 매수금액 277,000원 = 매수주가 55,400원 x 매도수량 5주
- 총매수금액 832,000원 = 555,000원 + 277,000원
- 총매도금액 867,000원 = 매도주가 57,800원 x 매도수량 15주 
- 매도세 1,995원 = 총매도금액 867,000원 x 0.23% 
- 매도수수료 163원 = 총매도금액 867,000원 x 0.015% + 총매도금액 867,000원 x 0.0036396% 
- 총실현수익 32,842원 = 총매도금액 867,000원 - 총매수금액 832,000원 - 매도세 1,995원 - 매도수수료 163원



평가손익 `UNREAL_PROFIT`은 보유주식의 현재가치에서 보유주식의 매수시 역사적가치를 차감하여 계산합니다.

In [15]:
r0 = df_result4.iloc[0]
r1 = df_result4.iloc[1]
r2 = df_result4.iloc[2]
print(f"""
- 보유주식의 현재가치 {r2.STOCK_VALUE:,}원 = 현재주가 {r2.PRICE:,}원 x 보유수량 {r2.POSITION:,}주
- 보유주식의 역사적가치 {r2.HIST_VALUE:,}원 = 2차매수주가 {r1.TRADE_PRICE:,}원 x 2차매수수량 중 잔여수량 15주
- 평가손익 {r2.UNREAL_PROFIT:,}원 = 현재가치 {r2.STOCK_VALUE:,}원 - 역사적가치 {r2.HIST_VALUE:,}원
""")



- 보유주식의 현재가치 867,000원 = 현재주가 57,800원 x 보유수량 15주
- 보유주식의 역사적가치 831,000원 = 2차매수주가 55,400원 x 2차매수수량 중 잔여수량 15주
- 평가손익 36,000원 = 현재가치 867,000원 - 역사적가치 831,000원



총손익 `PROFIT`은 최종매매일까지의 실현손익의 누적과 최종매매일의 평가손익의 합입니다.

In [16]:
r0 = df_result4.iloc[0]
r1 = df_result4.iloc[1]
r2 = df_result4.iloc[2]
all_real = r0.REAL_PROFIT + r1.REAL_PROFIT + r2.REAL_PROFIT
print(f"""
- {r0.DATE.strftime('%Y-%m-%d')} 실현손익 {r0.REAL_PROFIT:,}원
- {r1.DATE.strftime('%Y-%m-%d')} 실현손익 {r1.REAL_PROFIT:,}원
- {r2.DATE.strftime('%Y-%m-%d')} 실현손익 {r2.REAL_PROFIT:,}원
- 실현손익 누적 {all_real:,}원 = ({r0.REAL_PROFIT:,}원) + ({r1.REAL_PROFIT:,}원) + {r2.REAL_PROFIT:,}원
- {r2.DATE.strftime('%Y-%m-%d')} 평가손익 {r2.UNREAL_PROFIT:,}원
- {r2.DATE.strftime('%Y-%m-%d')} 총손익 {r2.PROFIT:,}원 = "{all_real:,}원 + {r2.UNREAL_PROFIT:,}원"
""")



- 2023-01-02 실현손익 -105원
- 2023-01-03 실현손익 -208원
- 2023-01-04 실현손익 32,842원
- 실현손익 누적 32,529원 = (-105원) + (-208원) + 32,842원
- 2023-01-04 평가손익 36,000원
- 2023-01-04 총손익 68,529원 = "32,529원 + 36,000원"



## 5.9 주문 오류 처리

백테스트 실행시 주문에 다음과 같은 오류사항이 있는 경우 경고가 발생하고 해당 주문은 무효처리됩니다.

- `KQuantNotAllowShort`: 공매도 경고. 보유하지 않은 주문을 매도하거나 보유한 수량보다 많은 수량을 매도 주문한 경우
- `KQuantNotAllowLoan`: 현금보유 경고. 보유하지 않은 주문을 매도하거나 보유한 수량보다 많은 수량을 매도 주문한 경우
- `KQuantInvalidSymbol`: 주식종목 경고. 종목코드가 잘못된 경우



In [17]:
df_order5 = pd.DataFrame({
    "DATE": ["2023-01-02", "2023-01-03", "2023-01-04", "2023-01-05"],
    "ORDER": [-10, 10, -20, 10]
})

df_order5

,DATE,ORDER
0,2023-01-02,-10
1,2023-01-03,10
2,2023-01-04,-20
3,2023-01-05,10


예를 들어 위 주문은 다음 오류를 발생합니다.

- 1월 2일 : 보유하지 않은 주식 10주를 매도하려고 하므로 공매도 경고가 발생하고 주문은 실행되지 않습니다.
- 1월 4일 : 보유주식이 10주인데 20주를 매도하려고 하므로 공매도 경고가 발생하고 주문은 실행되지 않습니다.
- 1월 5일 : 현금잔고보다 많은 금액의 주식을 매수하려고 하므로 현금보유 경고가 발생하고 주문은 실행되지 않습니다.


In [18]:
kq.backtest_stock_daily("005930", df_order5, init_cash=600_000)

[2023-01-02] <KQuantNotAllowShort> 공매도 금지 오류: 매도 수량이 현재 보유수량보다 큼
[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:    -10, 매매수량:      0, 주문후 보유수량:      0
[2023-01-03] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-04] <KQuantNotAllowShort> 공매도 금지 오류: 매도 수량이 현재 보유수량보다 큼
[2023-01-04] 종목: 005930, 주문전 보유수량:     10 주문수량:    -20, 매매수량:      0, 주문후 보유수량:     10
[2023-01-05] <KQuantNotAllowLoan> 융자 금지 오류: 매매가능 현금 미보유
[2023-01-05] 종목: 005930, 주문전 보유수량:     10 주문수량:     10, 매매수량:      0, 주문후 보유수량:     10


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,005930,"55,500",-10,0,0,0,0.0000,0,0,0,0,"600,000",0,0,"600,000",0,0,0,"600,000",0
1,2023-01-03,005930,"55,400",10,10,"55,400",10,"55,400.0000",0,0,0,"-554,000","46,000","554,000","554,000","600,000",0,0,0,"600,000",0
2,2023-01-04,005930,"57,800",-20,0,0,10,"55,400.0000",0,0,0,0,"46,000","554,000","578,000","624,000",0,"24,000","24,000","624,000",0
3,2023-01-05,005930,"58,200",10,0,0,10,"55,400.0000",0,0,0,0,"46,000","554,000","582,000","628,000",0,"28,000","28,000","628,000",0


:::{.callout-caution title="주의사항"}
오류 주문은 예외(Exception)가 아닌 경고(Warning)로 처리되어 해당 주문정보만 무시되고 나머지 정상적인 주문정보은 그대로 처리되는 점에 주의하시기 바랍니다.
:::

## 5.10 포워드테스트

시간이 지나면서 새로운 주가정보가 발생하였을 때 진행중인 전략의 백테스트를 계속 갱신하며 매매전략의 성능을 모니터링하는 것을 포워드테스트(forward-test)라고 합니다.

kquant는 포워드 테스트를 위해 과거의 백테스트에 새로운 주가정보와 주문정보를 추가하여 백테스트 결과를 갱신하는 기능을 `backtest_update_stock_daily` 함수로 지원합니다.

`backtest_update_stock_daily` 함수를 사용하기 위해서는 최초 백테스트를 수행할 때 다음 코드와 같이 `return_position` 인수를 `True`로 주어 현재의 주식 보유상태를 저장하는 `df_position` 데이터프레임을 추가로 출력해야 합니다.

In [19]:
df_order6 = pd.DataFrame({
    "DATE": ["2023-01-02", "2023-01-03"],
    "ORDER": [10, 10]
})

df_result6, df_position6 = kq.backtest_stock_daily(
    "005930", 
    df_order6, 
    init_cash=10_000_000, 
    return_position=True,
)


[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-03] 종목: 005930, 주문전 보유수량:     10 주문수량:     10, 매매수량:     10, 주문후 보유수량:     20


In [20]:
df_result6


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,005930,"55,500",10,10,"55,500",10,"55,500.0000",0,0,0,"-555,000","9,445,000","555,000","555,000","10,000,000",0,0,0,"10,000,000",0
1,2023-01-03,005930,"55,400",10,10,"55,400",20,"55,450.0000",0,0,0,"-554,000","8,891,000","1,109,000","1,108,000","9,999,000",0,"-1,000","-1,000","10,000,000","1,000"


In [21]:
df_position6


,DATE,SYMBOL,QTY,TRADE_PRICE,HIST_VALUE,FEE,NOT_DELETE
0,2023-01-02,005930,10,"55,500","555,000",0,True
1,2023-01-03,005930,10,"55,400","554,000",0,True


이렇게 계산한 `df_result` 및 `df_position` 데이터프레임을 새로운 주문 정보와 함께 `backtest_update_stock_daily` 함수에 넣어주면 백테스트 결과가 추가로 갱신됩니다.

이 때 `backtest_update_stock_daily` 함수에는 다음과 같은 인수를 주어야 합니다.

- `symbol`: 종목 단축코드 문자열
- `order`: 해당일의 주문 수량
- `date`: 매매일
- `df_result`: 과거의 백테스트 결과 데이터프레임
- `df_position`: 과거의 백테스트 포지션 데이터프레임


In [22]:
df_result7, df_position7 = kq.backtest_update_stock_daily(
    "005930", 
    -15, 
    "2023-01-04", 
    df_result6, 
    df_position6,
)


[2023-01-04] 종목: 005930, 주문전 보유수량:     20 주문수량:    -15, 매매수량:    -15, 주문후 보유수량:      5


In [23]:
df_result7


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,005930,"55,500",10,10,"55,500",10,"55,500.0000",0,0,0,"-555,000","9,445,000","555,000","555,000","10,000,000",0,0,0,"10,000,000",0
1,2023-01-03,005930,"55,400",10,10,"55,400",20,"55,450.0000",0,0,0,"-554,000","8,891,000","1,109,000","1,108,000","9,999,000",0,"-1,000","-1,000","10,000,000","1,000"
2,2023-01-04,005930,"57,800",-15,-15,"57,800",5,"55,400.0000",0,0,0,"867,000","9,758,000","277,000","289,000","10,047,000","35,000","12,000","47,000","10,047,000",0


In [24]:
df_position7


,DATE,SYMBOL,QTY,TRADE_PRICE,HIST_VALUE,FEE,NOT_DELETE
0,2023-01-03,005930,5,"55,400","831,000",0,True


## 5.11 전략 함수를 사용한 백테스트

`backtest_stock_daily` 함수의 `order` 인수는 실제 주문정보가 담긴 데이터프레임 이외에도 주문정보를 생성할 수 있는 전략 함수를 받을 수도 있습니다. 

`order` 인수로 넣을 수 있는 함수는 반드시 다음과 같은 인수를 가져야 합니다.

- `symbol`: 종목단축코드
- `date`: datetime.date 형식의 날짜 정보
- `df_result`: 백테스트 결과 데이터프레임
- `df_position`: 백테스트 포지션 데이터프레임
- `logger`: 로거 객체

다음 예제 코드는 주가가 2 매매일 전보다 3% 이상 상승하면 매수하고 일단 매매한 주식의 손익 절대값이 +/- 3% 이상이면 매도하는 전략을 함수로 구현한 것입니다.

In [25]:
import pandas as pd
import datetime as dt
import logging

def trade_func(
    symbol: str,
    date: dt.date,
    df_result: pd.DataFrame,
    df_position: pd.DataFrame,
    logger: logging.Logger,
) -> int:
    
    # 보유주식이 없는 경우
    if len(df_position) == 0:
        
        # 2일간 주가 수익률 계산
        start_date = date - dt.timedelta(days=10) 
        price = kq.daily_stock(symbol, start_date, date).CLOSE
        price_cur = price.iloc[-1]
        price_2daysago = price.iloc[-3]
        return_2days = price_cur / price_2daysago - 1

        if return_2days > 0.03:
            return 1
        else:
            return 0
        
    # 보유주식이 있는 경우
    else:
        
        # 수익률 계산
        trade_price = df_position.iloc[-1].TRADE_PRICE
        profit = df_result.iloc[-1].PROFIT
        profit_pct = profit / trade_price
        
        if abs(profit_pct) > 0.03:
            return -df_position.iloc[-1].QTY
        else:
            return 0
    



백테스트 결과는 다음과 같습니다.

In [26]:
df_result8 = kq.backtest_stock_daily(
    "005930", 
    trade_func,
    "2023-01-02", 
    "2023-01-31", 
    init_cash=100_000,
)

[2023-01-04] 종목: 005930, 주문전 보유수량:      0 주문수량:      1, 매매수량:      1, 주문후 보유수량:      1
[2023-01-10] 종목: 005930, 주문전 보유수량:      1 주문수량:     -1, 매매수량:     -1, 주문후 보유수량:      0
[2023-01-25] 종목: 005930, 주문전 보유수량:      0 주문수량:      1, 매매수량:      1, 주문후 보유수량:      1
[2023-01-26] 종목: 005930, 주문전 보유수량:      1 주문수량:     -1, 매매수량:     -1, 주문후 보유수량:      0


In [27]:
kq.backtest_plot_stock_daily(df_result8)


## 5.12 전략 함수를 사용한 포워드테스트

전략함수를 사용하여 포워드테스트를 하는 경우에는 
전략 함수 객체를 `backtest_update_stock_daily` 함수에 바로 넣을 수 없으므로
사용자가 전략함수를 실행하여 주문 수량을 계산한 뒤 이 값을 넣어야 합니다.

일단 전략함수를 인수로 넣고 백테스트를 수행합니다. 이 때 `return_logger` 인수를 True로 넣으면  전략함수에 넣을 로거 객체를 반환합니다.


In [28]:
df_result9, df_position9, logger = kq.backtest_stock_daily(
    "005930",
    trade_func, 
    "2023-01-02",
    "2023-01-03",
    init_cash=100_000, 
    return_position=True,
    return_logger=True,
)

다음으로 사용자가 직접 매매수량을 계산합니다.

In [29]:
from dateutil.parser import parse

date = parse("2023-01-04")
order_qty = trade_func(
    "005930", 
    date, 
    df_result9, 
    df_position9, 
    logger,
)

order_qty

1

계산한 매매수량과 과거 백테스트 데이터프레임을 `backtest_update_stock_daily` 함수에 입력하여 포워드테스트를 수행합니다.

In [30]:

df_result10, df_position10 = kq.backtest_update_stock_daily(
    "005930",
    order_qty,
    date,
    df_result9,
    df_position9,
)


[2023-01-04] 종목: 005930, 주문전 보유수량:      0 주문수량:      1, 매매수량:      1, 주문후 보유수량:      1


앞서 수행한 백테스트와 동일한 결과가 나오는 것을 확인할 수 있습니다.

In [31]:
df_result8.iloc[:3].equals(df_result10)

True

같은 방식으로 매일 포워드테스트를 진행할 수 있습니다.